In [1]:
import pandas as pd
import plotly.express as px
import os 


In [5]:
test_parquet=os.path.join(os.getcwd(), 'data', 'asl-signs', 'train_landmark_files','2044', "635217.parquet")
df= pd.read_parquet(test_parquet) #this is on the dataset

In [6]:
df.columns

Index(['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'], dtype='object')

In [7]:
df

,frame,row_id,type,landmark_index,x,y,z
0,22,22-face-0,face,0,0.438251,0.449453,-0.047826
1,22,22-face-1,face,1,0.414527,0.404880,-0.071994
2,22,22-face-2,face,2,0.423745,0.420681,-0.042145
3,22,22-face-3,face,3,0.402349,0.372041,-0.043906
4,22,22-face-4,face,4,0.411857,0.393013,-0.074747
...,...,...,...,...,...,...,...
3796,28,28-right_hand-16,right_hand,16,0.237886,0.460492,-0.121701
3797,28,28-right_hand-17,right_hand,17,0.163785,0.452878,-0.134627
3798,28,28-right_hand-18,right_hand,18,0.279028,0.476364,-0.156879
3799,28,28-right_hand-19,right_hand,19,0.252220,0.497005,-0.139577


In [8]:
px.scatter_3d(df, x="x", y="y", z="z" , opacity=0.5, width=800, height=800).show()

In [2]:
df2=pd.read_parquet("2d_landmarks.parquet") #rgb image landmarks
df2.columns

Index(['type', 'landmark_index', 'frame', 'x', 'y', 'z'], dtype='object')

In [3]:
df2

,type,landmark_index,frame,x,y,z
0,face,0,1,NaN,NaN,NaN
1,face,1,1,NaN,NaN,NaN
2,face,2,1,NaN,NaN,NaN
3,face,3,1,NaN,NaN,NaN
4,face,4,1,NaN,NaN,NaN
...,...,...,...,...,...,...
8140,right_hand,16,15,NaN,NaN,NaN
8141,right_hand,17,15,NaN,NaN,NaN
8142,right_hand,18,15,NaN,NaN,NaN
8143,right_hand,19,15,NaN,NaN,NaN


In [4]:
px.scatter_3d(df2, x="x", y="y", z="z" , opacity=0.5, width=800, height=800).show()

In [10]:
df3= pd.read_parquet("3d_landmarks.parquet") #rgb image landmarks

In [11]:
px.scatter_3d(df3, x="x", y="y", z="z" , opacity=0.5, width=800, height=800).show()

In [ ]:
import os  #this was a test to see openpifpaf plotting on a specific image. 
import openpifpaf
from PIL import Image
import matplotlib as plt
import numpy as np
from openpifpaf import show


# Path setup
wd = os.getcwd()
img_path = os.path.join(wd, 'temp', 'asl_shh.jpg')
img_pil = Image.open(img_path).convert("RGB")
img_np = np.array(img_pil)

# List of models to run
checkpoints = [
    #'resnet50',
    #'shufflenetv2k30',
    #'shufflenetv2k16',
    'shufflenetv2k16-wholebody',
    'shufflenetv2k30-wholebody'
]

for ckpt in checkpoints:
    print(f"🔄 Running model: {ckpt}")
    predictor = openpifpaf.Predictor(checkpoint=ckpt)
    predictions, _, _ = predictor.numpy_image(img_np)

    painter = show.AnnotationPainter()

    out_path = os.path.join(wd, 'temp', f'asl_shh_keypoints_{ckpt.replace("-", "_")}.jpg')

    with show.Canvas.image(img_np) as ax:
        painter.annotations(ax, predictions)
        ax.axis('off')
        ax.set_title(f'Model: {ckpt}')
        ax.set_xlim(0, img_np.shape[1])
        ax.set_ylim(img_np.shape[0], 0)
        fig = ax.figure
        fig.tight_layout()
        fig.canvas.draw()
        fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
        print(f"✅ Saved: {out_path}")

🔄 Running model: shufflenetv2k16-wholebody


c:\Users\kabir\.vscode\Code Files\librealsense_cv\.venv\lib\site-packages\ipykernel_launcher.py:40: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


✅ Saved: c:\Users\kabir\.vscode\Code Files\librealsense_cv\rgb_vs_pointcloud\temp\asl_shh_keypoints_shufflenetv2k16_wholebody.jpg
🔄 Running model: shufflenetv2k30-wholebody


c:\Users\kabir\.vscode\Code Files\librealsense_cv\.venv\lib\site-packages\ipykernel_launcher.py:40: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


✅ Saved: c:\Users\kabir\.vscode\Code Files\librealsense_cv\rgb_vs_pointcloud\temp\asl_shh_keypoints_shufflenetv2k30_wholebody.jpg


In [ ]:
import os #this was an attempt to isolate openpifpaf keypoints for face and hands.
import numpy as np
import openpifpaf
from PIL import Image
import matplotlib.pyplot as plt
from openpifpaf.plugins.wholebody.constants import WHOLEBODY_KEYPOINTS

# Setup
wd = os.getcwd()
img_path = os.path.join(wd, 'temp', 'asl_shh.jpg')
out_path = os.path.join(wd, 'temp', 'asl_shh_facehands_fixed.jpg')
#img_path = os.path.join(wd, 'temp', 'listen.jpg')
#out_path = os.path.join(wd, 'temp', 'listen_kp.jpg')
# Load + predict
img_np = np.array(Image.open(img_path).convert("RGB"))
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k30-wholebody')
predictions, _, _ = predictor.numpy_image(img_np)
# Slice correctly from full keypoint list
kp = predictions[0].data

face_ids = list(range(17, 85))             # 68 total
left_hand_ids = list(range(91, 112))       # 21
right_hand_ids = list(range(112, 133))     # 21

face_kp = kp[face_ids]
left_kp = kp[left_hand_ids]
right_kp = kp[right_hand_ids]
left_index_kp = left_kp[5:9]
right_index_kp = right_kp[5:9]

# Plot
fig, ax = plt.subplots()
ax.imshow(img_np)
ax.axis('off')

# Face: orange
ax.scatter(face_kp[face_kp[:,2]>0][:,0], face_kp[face_kp[:,2]>0][:,1], c='orange', s=20, label='Face')

# Left hand: blue
ax.scatter(left_kp[left_kp[:,2]>0][:,0], left_kp[left_kp[:,2]>0][:,1], c='blue', s=20, label='Left Hand')

# Right hand: green
ax.scatter(right_kp[right_kp[:,2]>0][:,0], right_kp[right_kp[:,2]>0][:,1], c='green', s=20, label='Right Hand')
ax.scatter(left_index_kp[left_index_kp[:, 2] > 0][:, 0],
           left_index_kp[left_index_kp[:, 2] > 0][:, 1],
           c='blue', s=20)

ax.scatter(right_index_kp[right_index_kp[:, 2] > 0][:, 0],
           right_index_kp[right_index_kp[:, 2] > 0][:, 1],
           c='green', s=20)
ax.legend()
fig.savefig(out_path, bbox_inches='tight', pad_inches=0.1, dpi=300)
plt.close()
print(f"✅ Full face (including mouth) saved to {out_path}")



✅ Full face (including mouth) saved to c:\Users\kabir\.vscode\Code Files\librealsense_cv\rgb_vs_pointcloud\temp\listen_kp.jpg
